In [2]:
'''
Filter fasta and convert annotation Step by Step
Research result: 
Engineering Result: a suite that cut fasta and update annotation
'''
from Bio import SeqIO
from operator import add

In [41]:
gene_locs = [(12158,12994), (15855, 16226), (28738,29638),(2288689,2289986),(2995039,2995770)]

In [42]:
Schpo1Fasta = SeqIO.parse('/home/richard/research/tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1_chrome1.fasta','fasta')
Schpo1Fasta = list(Schpo1Fasta)[0]

#### First look at start, stop locations with positive and negative strand

In [54]:
# for a,b in gene_locs:
#     print Schpo1Fasta[a-1:a+2], Schpo1Fasta[b-3:b]
#     print

for a,b in gene_locs:
    print Schpo1Fasta[a-1:b]
    print

ID: chromosome_1
Name: chromosome_1
Description: chromosome_1
Number of features: 0
Seq('ATGTCCCCTTTAATAGTTGGGACTTTAATAATTATCCTATTGTCAGGACTCGCA...TAA', SingleLetterAlphabet())

ID: chromosome_1
Name: chromosome_1
Description: chromosome_1
Number of features: 0
Seq('ATGTCCCCTTTAATAGTTGGGACTTTAATAATTATCCTATTGTCAGGACTCGCA...TAA', SingleLetterAlphabet())

ID: chromosome_1
Name: chromosome_1
Description: chromosome_1
Number of features: 0
Seq('ATGTCAAATCCAGAAAGCTTGAAAAAACAGGTTGAACCTCCTGGTTACAATGAG...TAA', SingleLetterAlphabet())

ID: chromosome_1
Name: chromosome_1
Description: chromosome_1
Number of features: 0
Seq('TTAAAGATGCCTGATGTGTTGAGAAACAGTCTCGTATACATAAAATGTAACAGC...CAT', SingleLetterAlphabet())

ID: chromosome_1
Name: chromosome_1
Description: chromosome_1
Number of features: 0
Seq('TTACTTAAACCAAGGTTTGGTTTCTTGAGCTTTCATAACTGAGTTGCGTAGTCC...CAT', SingleLetterAlphabet())



##From the Above, I realize that the anti-sense part of the DNA is actually 'reverse sense', which means if we read it in the reversed way, we will get a sense sequence

#### 1. Cut genomes

In [74]:
def cut_genome(seq_record, a, b, intergenic_length):
    '''
    input: SeqRecord; start and stop location (a,b) on annotation(index starts from 1)
    output: SeqRecord with the desired region and an intergenic region at front
    '''
    gene = seq_record[a-1:b]
    
    if a-1-intergenic_length > 0:
        intergenic_region_front = seq_record[a-1-intergenic_length:a-1]
    else:
        intergenic_region_front = seq_record[b:b+intergenic_length]
        
#     if b+intergenic_length < len(seq_record):
#         intergenic_region_back = seq_record[a-1-intergenic_length:a-1]
#     else:
#         intergenic_region_front = seq_record[b:b+intergenic_length]
    return intergenic_region_front+gene

In [75]:
a,b = (12158,12994)
gene = cut_genome(Schpo1Fasta, a, b, 100)

In [77]:
gene[100:]

SeqRecord(seq=Seq('ATGTCCCCTTTAATAGTTGGGACTTTAATAATTATCCTATTGTCAGGACTCGCA...TAA', SingleLetterAlphabet()), id='chromosome_1', name='chromosome_1', description='chromosome_1', dbxrefs=[])

In [79]:
genes = map(lambda (a,b): cut_genome(Schpo1Fasta, a, b, 100), gene_locs)

In [82]:
Schpo1FastaFiveGeneCuts = reduce(add, genes)+genes[0][:100]

In [85]:
target = '/home/richard/research/tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1FiveGenes.fasta'
SeqIO.write(Schpo1FastaFiveGeneCuts, target, 'fasta')

1

In [ ]:
def cut_genomes(seq_record, gene_locs, intergenic_length):
    genes = map(lambda (a,b): cut_genome(seq_record, a, b, intergenic_length), gene_locs)
    return reduce(add, genes)+genes[0][:100]

In [145]:
gene_locs

[('"SPAC212.08c";', (12158, 12994)),
 ('"SPAC212.12";', (15855, 16226)),
 ('"SPAC212.01c";', (28738, 29638)),
 ('"SPAC4G9.20c";', (2288689, 2289986)),
 ('"SPAC31G5.05c";', (2995039, 2995770))]

In [154]:
def cut_single_fasta(fastaFile, fastaFileCut, gene_location, intergenic_length):
    fasta = SeqIO.parse(fastaFile,'fasta')
    seq_record = list(fasta)[0]
    
    seq_record_cut = cut_genes(seq_record, gene_location, intergenic_length)
    SeqIO.write(seq_record_cut, fastaFileCut, 'fasta')
    print 'The cut fasta file is saved at', fastaFileCut

In [156]:
fastaFile = '/home/richard/research/tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1_chrome1.fasta'
fastaFileCut = '/home/richard/research/tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1_CutTest.fasta'
intergenic_length = 100
cut_single_fasta(fastaFile, fastaFileCut, gene_locs, intergenic_length)

The cut fasta file is saved at /home/richard/research/tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1_CutTest.fasta


### Comment: remember to sort (start, end) when dealing with annotation

#### 2. Update annotation to match genomes that are cut

In [152]:
SeqIO.write(cut_genes(Schpo1Fasta, gene_locs, intergenic_length),'../tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1FiveGenesV2.fasta','fasta')

1

In [3]:
def cut_gene(seq_record, a, b, intergenic_length):
    '''
    helper function
    input: SeqRecord; start and stop location (a,b) on annotation(index starts from 1)
    output: SeqRecord with the desired region and an intergenic region at front
    '''
    gene = seq_record[a-1:b]
    
    if a-1-intergenic_length > 0:
        intergenic_region_front = seq_record[a-1-intergenic_length:a-1]
    else:
        intergenic_region_front = seq_record[b:b+intergenic_length]
        
    return intergenic_region_front+gene

def cut_genes(seq_record, gene_location, intergenic_length):
    '''
    seq_record should be a single genome
    batch cut
    '''
    gene_location = [atuple for (key, atuple) in gene_location]
    genes = map(lambda (a,b): cut_gene(seq_record, a, b, intergenic_length), gene_location)
    return reduce(add, genes)+genes[0][:intergenic_length]

def cut_single_fasta(fastaFile, fastaFileCut, gene_location, intergenic_length):
    fasta = SeqIO.parse(fastaFile,'fasta')
    seq_record = list(fasta)[0]
    
    seq_record_cut = cut_genes(seq_record, gene_location, intergenic_length)
    SeqIO.write(seq_record_cut, fastaFileCut, 'fasta')
    print 'The cut fasta file is saved at', fastaFileCut


def get_gene_location(inputFile):
    '''
    helper function for 'update_annotation'
    input: annotation file
    output: tuple, (genename, range)
    '''
    pairs = {} # put start, stop location to a list associated with 
    with open(inputFile) as f:
        for line in f:
            if 'start_codon' in line or 'stop_codon' in line:
                alist = line.split()
                a,b = map(int,[alist[3], alist[4]])
                key = alist[9]
                if key not in pairs:
                    pairs[key] = [a,b]
                else:
                    pairs[key].append(a); pairs[key].append(b)
                    
    for key in pairs:
        pairs[key] = sorted(pairs[key])
        
    gene_locs = map(lambda (key, alist): (key, (min(alist), max(alist))), pairs.items())
    return sorted(gene_locs, key = lambda x: x[1])

def rearrange_location(gene_locs, intergenic_length):
    '''
    helper function for 'update_annotation'
    '''
    acc = 0
    shift = {}
    for pairs in gene_locs:
        acc += intergenic_length
        key,[a,b] = pairs
        shift[key] = acc-a+1
        gene_length = b-a+1
        acc+=gene_length
    return shift


def update_annotation(inputFile, outputFile, gene_locs, intergenic_length):
    shift = rearrange_location(gene_locs, intergenic_length)
    
    with open(inputFile) as f:
        with open(outputFile,'w') as f1:
            for line in f:
                alist1 = line.split()
                key = alist1[9]
                alist2 = line.split('\t')
                start = shift[key]+int(alist1[3]); stop = shift[key]+int(alist1[4])
                if start < 0: continue
                alist2[3] = str(start); alist2[4] = str(stop)
                f1.write('\t'.join(alist2))
    print 'The updated annotation file is saved at:',outputFile
    
    
def update_annotation_cut_fasta(intergenic_length, inputFile, outputFile, fastaFile, fastaFileCut):
    gene_locs = get_gene_location(inputFile)
    
    update_annotation(inputFile, outputFile, gene_locs, intergenic_length)
    
    cut_single_fasta(fastaFile, fastaFileCut, gene_locs, intergenic_length)

In [144]:
inputFile = '../tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1FiveGenesShuffled.gtf'
outputFile = '../tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1FiveGenesShuffledUpdatedV2.gtf'
update_annotation(inputFile, outputFile, 100)
# gene_locs = get_gene_location(inputFile)
# shift = rearrange_location(gene_locs, 100)

The updated annotation file is saved at: ../tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1FiveGenesShuffledUpdatedV2.gtf


In [134]:
with open(inputFile) as f:
    with open(outputFile,'w') as f1:
        for line in f:
            alist1 = line.split()
            key = alist1[9]
    #         print alist[:3], shift[key]+int(alist[3]), shift[key]+int(alist[4])
            alist2 = line.split('\t')
            start = shift[key]+int(alist1[3]); stop = shift[key]+int(alist1[4])
            if start < 0: continue
            alist2[3] = str(start); alist2[4] = str(stop)
            f1.write('\t'.join(alist2))

In [108]:
get_gene_location(inputFile)

[('"SPAC212.08c";', (12158, 12994)),
 ('"SPAC212.12";', (15855, 16226)),
 ('"SPAC212.01c";', (28738, 29638)),
 ('"SPAC4G9.20c";', (2288689, 2289986)),
 ('"SPAC31G5.05c";', (2995039, 2995770))]

In [ ]:
gene_locs = [(12158,12994), (15855, 16226), (28738,29638),(2288689,2289986),(2995039,2995770)]

In [105]:
inputFile = '../tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1FiveGenesShuffled.gtf'

with open(inputFile) as f:
    pairs = {}
    for line in f:
        if 'start_codon' in line or 'stop_codon' in line:
            alist = line.split()
            a,b = map(int,[alist[3], alist[4]])
            key = alist[9]
            if key not in pairs:
                pairs[key] = [a,b]
            else:
                pairs[key].append(a); pairs[key].append(b)
    for key in pairs:
        pairs[key] = sorted(pairs[key])
    
print map(lambda (key, alist): (key, (min(alist), max(alist))), pairs.items())
gene_locs = map(lambda (key, alist): (key, (min(alist), max(alist))), pairs.items())
sorted(gene_locs, key = lambda x: x[1])
# print sorted

[('"SPAC212.08c";', (12158, 12994)), ('"SPAC212.12";', (15855, 16226)), ('"SPAC212.01c";', (28738, 29638)), ('"SPAC4G9.20c";', (2288689, 2289986)), ('"SPAC31G5.05c";', (2995039, 2995770))]


[('"SPAC212.08c";', (12158, 12994)),
 ('"SPAC212.12";', (15855, 16226)),
 ('"SPAC212.01c";', (28738, 29638)),
 ('"SPAC4G9.20c";', (2288689, 2289986)),
 ('"SPAC31G5.05c";', (2995039, 2995770))]

#### 3. Combine cut annotation and fasta together

In [157]:
def update_annotation_cut_fasta(intergenic_length, inputFile, outputFile, fastaFile, fastaFileCut):
    gene_locs = get_gene_location(inputFile)
    
    update_annotation(inputFile, outputFile, gene_locs, intergenic_length)
    
    cut_single_fasta(fastaFile, fastaFileCut, gene_locs, intergenic_length)

In [158]:
inputFile = '../tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1FiveGenesShuffled.gtf'
outputFile = '../tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1FiveGenesShuffledUpdatedV3.gtf'
fastaFile = '/home/richard/research/tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1_chrome1.fasta'
fastaFileCut = '/home/richard/research/tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1_chrome1_CutV3.fasta'
intergenic_length = 100

In [161]:
update_annotation_cut_fasta(intergenic_length, inputFile, outputFile, fastaFile, fastaFileCut)

The updated annotation file is saved at: ../tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1FiveGenesShuffledUpdatedV3.gtf
The cut fasta file is saved at /home/richard/research/tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1_chrome1_CutV3.fasta


In [170]:
inputFile = '../tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1_converted_fixed.gtf'
outputFile = '../tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1_converted_fixed.updated.gtf'
fastaFile = '/home/richard/research/tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1_chrome1.fasta'
fastaFileCut = '/home/richard/research/tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1_chrome1.cut.fasta'
intergenic_length = 300
update_annotation_cut_fasta(intergenic_length, inputFile, outputFile, fastaFile, fastaFileCut)

The updated annotation file is saved at: ../tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1_converted_fixed.updated.gtf
The cut fasta file is saved at /home/richard/research/tests/Oct22CutGenomeByAnnotationReDesign/_Schpo1Test/Schpo1_chrome1.cut.fasta


In [4]:
inputFile = '../tests/Oct23TestCutSuite/NCrassa/NCrassaContig12.1.gtf'
outputFile = '../tests/Oct23TestCutSuite/NCrassa/NCrassaContig12.1.updated.gtf'
fastaFile = '../tests/Oct23TestCutSuite/NCrassa/Neucr2_Supercontig_1.fasta'
fastaFileCut = '../tests/Oct23TestCutSuite/NCrassa/Neucr2_Supercontig_1.updated.fasta'
intergenic_length = 100
update_annotation_cut_fasta(intergenic_length, inputFile, outputFile, fastaFile, fastaFileCut)

The updated annotation file is saved at: ../tests/Oct23TestCutSuite/NCrassa/NCrassaContig12.1.updated.gtf
The cut fasta file is saved at ../tests/Oct23TestCutSuite/NCrassa/Neucr2_Supercontig_1.updated.fasta
